In [ ]:
pip install -U langgraph "langchain[anthropic]"


In [ ]:
import os
# Set the ANTHROPIC_API_KEY environment variable
os.environ["ANTHROPIC_API_KEY"] = "" #Add your API key here


# Create a ReAct Agent

In [ ]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='de823de3-5555-4576-9dae-aa7028cd66b3'),
  AIMessage(content=[{'text': 'I can help you get the weather for San Francisco. Let me check that for you.', 'type': 'text'}, {'id': 'toolu_01TQ85oE221xv4CTuwFZno33', 'input': {'city': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01J49hbrMBBAVgY6Jvjjtuks', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 386, 'output_tokens': 73, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--72b08df6-9e97-42d9-97b0-5c0fdb34df51-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'toolu_01TQ85oE221xv4CTuwFZno33', 'type': 'tool_call'}], usage_metadata={'input_tokens': 

# Configure LLM with specific parameters such as temperature

In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

model = init_chat_model(
    "anthropic:claude-3-7-sonnet-latest",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
)

# Add a system prompt

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather. If asked, tell the user explicitly that I cannot answer questions about the weather."
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='bf47f5cd-44f5-4d7d-8987-5095297bff56'),
  AIMessage(content="I cannot answer questions about the weather. While I do have access to a weather tool, I'm restricted from providing weather information as per my guidelines. I apologize for any inconvenience this may cause.\n\nIf there's something else I can help you with that doesn't involve weather information, please let me know.", additional_kwargs={}, response_metadata={'id': 'msg_01GwnydEVodxZHhekHt4UV5e', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 404, 'output_tokens': 68, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--8e518f17-cf09-4c82-8d05-b13727a5f1b0-0', usage_metadata={'input_tokens': 404, 'output_tokens': 68, 'total_tokens'

# Add memory to your agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    checkpointer=checkpointer
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

print(ny_response)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='0f71e142-851b-405f-84bc-351190bad85e'), AIMessage(content=[{'text': 'I can help you get the weather for San Francisco. Let me check that for you.', 'type': 'text'}, {'id': 'toolu_01LK98uGjumRopjjR79Lvp9m', 'input': {'city': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01EMu16kzSoFwgEPcAjGUzcs', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 380, 'output_tokens': 73, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--e84cedbd-7c20-4b7d-a5c5-ad17cdf7e4f3-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'toolu_01LK98uGjumRopjjR79Lvp9m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 38

# Return Structured Output

In [ ]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    response_format=WeatherResponse
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]

WeatherResponse(conditions='sunny')